# 数据质量分析
* 命令行运行时，切换工作目录

```
import os
os.chdir('./code')
sys.path.append(os.getcwd())

```

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer
from sklearn.impute import IterativeImputer
from scipy.stats import entropy
from scipy.stats import ks_2samp
from sklearn import metrics
from eda_kit import conditional_entropy,roc_auc_score,auc


In [2]:
A_train = pd.read_csv('../data/A_train.csv')
B_train = pd.read_csv('../data/B_train.csv')
B_test = pd.read_csv('../data/B_test.csv')

a_data = A_train
b_data = B_train
test_data = B_test

print (a_data.dtypes[a_data.dtypes == np.int64])
print (b_data.dtypes[b_data.dtypes == np.int64])

* 缺失值分析
    * https://blog.csdn.net/weixin_40159138/article/details/89421014
    * https://www.jianshu.com/p/9c867fb9cf17
    * https://scikit-learn.org/stable/modules/impute.html#impute
    * https://blog.csdn.net/qq_38958113/article/details/98220246
    * 从下图可以看到缺失值非常接近
    * B_test和A_train缺失值情况几乎一致
    * B_train的缺失值情况比A_train严重很多
    * A_train中20%的用户缺失维度在100个以内，40%的缺失维度在450个以上，60%用户的缺失维度在150个以内
    * B_train 和 B_test 38%的用户缺失维度在186左右，60%用户维度缺失在460以上
    * 缺失值在学习过程中，feature_importance 会降低，并不一定会影响学习效果，*可以对比是否填充缺失值对结果的影响*
    * 填充策略：
        * 离散值填充
        * 连续值填充
        * A的情况是正确的情况下，可以用于填充B的值，或者AB的分布，可以填充B_test的值
        * 对待缺失值不同的态度，为决定不同的填充策略：比如用户主动不填收入，那么这个null值应该赋予一个已有值之外的，比如特殊的-999
        * 对A值采取固定填充，但是可以对B采取transform的填充，
        * 填充时机？

In [ ]:
fea_null = np.sum(a_data.isnull(), axis=0)
feb_null = np.sum(b_data.isnull(), axis=0)
fet_null = np.sum(test_data.isnull(), axis=0)

plt.subplot(311).plot(fea_null.values)
plt.subplot(312).plot(feb_null.values)
plt.subplot(313).plot(fet_null.values)

# sort_values
plt.subplot(311).plot(np.sort(fea_null))
plt.subplot(312).plot(np.sort(feb_null))
plt.subplot(313).plot(np.sort(fet_null))
plt.title('feature null compare')
plt.show()

# 缺失值归一化
plt.plot(np.sort(fea_null/a_data.shape[0]), color='green')
plt.plot(np.sort(feb_null/b_data.shape[0]), color='blue')
plt.plot(np.sort(fet_null/test_data.shape[0]),color='red')
plt.show()

u_fea_null = np.sum(a_data.isnull(), axis=1)
u_feb_null = np.sum(b_data.isnull(), axis=1)
u_fet_null = np.sum(test_data.isnull(), axis=1)
u_fea_null.hist(cumulative=True, density=True, bins=100, alpha=.2, color="r")
u_feb_null.hist(cumulative=True, density=True, bins=100, alpha=.2, color="b")
u_fet_null.hist(cumulative=True, density=True, bins=100, alpha=.2, color="g")
plt.title('user feature null')
plt.show()
print( "用户的维度缺失情况")
print( (u_fea_null[u_fea_null<156].count()-0.0)/u_fea_null.count())
print( (u_feb_null[u_feb_null<186].count()-0.0)/u_feb_null.count())
print( (u_fet_null[u_fet_null<186].count()-0.0)/u_fet_null.count())

# 保留null值更少的维度
threshold_column_null = 0.99
a_less_null_columns = fea_null[fea_null<a_data.shape[0]*threshold_column_null].sort_values(axis=0).index
b_less_null_columns = feb_null[feb_null<b_data.shape[0]*threshold_column_null].sort_values(axis=0).index
test_less_null_columns = fet_null[fet_null<test_data.shape[0]*threshold_column_null].sort_values(axis=0).index
a_b_less_null_columns = a_less_null_columns.intersection(b_less_null_columns)


* 连续与离散
    * 知识：https://blog.csdn.net/ztf312/article/details/53991329
    * https://blog.csdn.net/banbuduoyujian/article/details/53957653
    * https://scikit-learn.org/stable/auto_examples/preprocessing/plot_discretization.html
    * https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.cut.html
    * https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing
    * https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-discretization
    * 489维数据中，383维的取值在100个以内，推测大部分应该是离散值
    * 取值在100个以上的可能为连续值
    * 取值大于300的维度81个，大于500的69个，这些极可能是连续值
    * 需要对连续值进行离散化：分类

In [ ]:
# 维度的取值个数，可以基本推断是连续值还是离散值
discrete_threshold = 100
#对所有维度的所有取值计算取值数量
tmp_all_data = pd.concat([a_data.sort_index(axis=1).drop(['flag','no'],axis=1),b_data.sort_index(axis=1).drop(['flag','no'],axis=1)])
tmp_all_data = pd.concat([tmp_all_data, test_data.sort_index(axis=1).drop(['no'],axis=1)])
all_data_n_unique = tmp_all_data.nunique()
all_discrete_columns = all_data_n_unique[all_data_n_unique<=discrete_threshold].index
all_continuous_columns = all_data_n_unique[all_data_n_unique>discrete_threshold].index
plt.hist(all_data_n_unique, cumulative=True, histtype='step', bins=100);plt.title('all data unique number');plt.show()


# 有很多维度的取值个数只有0 或者 1，这些维度是没有太大意义的。可以通过离散值条件熵删选，这里先去掉这部分
# 为什么有取值为0的？？？
b_data_n_unique = b_data.nunique()
b_discrete_columns = b_data_n_unique[b_data_n_unique<=discrete_threshold].index
b_continuous_columns = b_data_n_unique[b_data_n_unique>discrete_threshold].index
b_discrete_columns = b_discrete_columns.drop(b_data_n_unique[b_data_n_unique<=1].index)
plt.hist(b_data_n_unique, cumulative=True, histtype='step', bins=100);plt.title('B_train unique number');plt.show()
plt.hist(b_data_n_unique, cumulative=False, histtype='bar', bins=100, range=[0,2]);plt.title('B_train unique number');plt.show()


a_data_n_unique = a_data.nunique()
a_discrete_columns = a_data_n_unique[a_data_n_unique<=discrete_threshold].index
a_continuous_columns = a_data_n_unique[a_data_n_unique>discrete_threshold].index
a_discrete_columns = a_discrete_columns.drop(a_data_n_unique[a_data_n_unique<=1].index)
plt.hist(a_data_n_unique, cumulative=True, histtype='step', bins=100);plt.title('A_train unique number');plt.show()



all_data_n_unique.describe()
all_data_n_unique[b_continuous_columns].describe()
all_data_n_unique[a_continuous_columns].describe()


* 数据规范化
    * https://blog.csdn.net/weixin_38706928/article/details/80329563
    * https://scikit-learn.org/stable/modules/preprocessing.html

In [ ]:

b_scaler = StandardScaler().fit(b_data[b_continuous_columns])
a_scaler = StandardScaler().fit(a_data[a_continuous_columns])

a_standard_data = a_data
b_standard_data = b_data
test_standard_data = test_data
a_standard_data[a_continuous_columns] = a_scaler.transform(a_data[a_continuous_columns])
b_standard_data[b_continuous_columns] = b_scaler.transform(b_data[b_continuous_columns])
test_standard_data[b_continuous_columns] = b_scaler.transform(test_data[b_continuous_columns])

# 填充null值
fillna_b_ = 10
a_standard_fillna_data = a_standard_data.fillna(fillna_b_)
b_standard_fillna_data = b_standard_data.fillna(fillna_b_)
test_standard_fillna_data = test_standard_data.fillna(fillna_b_)

* 线性相关性
    * 剔除掉线性相关性大的维度
    * https://towardsdatascience.com/feature-selection-with-pandas-e3690ad8504b

In [ ]:
threshold_corr_bt = 0.98
corr_bt = b_standard_fillna_data.corr()
corr_length = corr_bt.shape[0]
final_cols_bt = []
del_cols_bt =[]
for i in range(corr_length):
    if corr_bt.columns[i] not in del_cols_bt:
        final_cols_bt.append(corr_bt.columns[i])
        for j in range(i+1,corr_length):
            if (corr_bt.iloc[i,j] > threshold_corr_bt) and (corr_bt.columns[j] not in del_cols_bt):
                print(i,j)
                del_cols_bt.append(corr_bt.columns[j])
b_less_corr_columns = a_b_less_null_columns.intersection(pd.Index(final_cols_bt))
b_less_corr_continuous_columns = b_less_corr_columns.intersection(b_continuous_columns)
b_less_corr_discrete_columns = b_less_corr_columns.intersection(b_discrete_columns)
sns.heatmap(b_standard_fillna_data[b_less_corr_columns].corr(), annot=False, cmap=plt.cm.Reds);plt.title('B correlation');plt.show()


* 同分布检验
    * https://blog.csdn.net/qq_41679006/article/details/80977113
    * https://www.cnblogs.com/arkenstone/p/5496761.html
    * https://blog.csdn.net/t15600624671/article/details/78770239
    * B_test 和 B_train只有2维数据的分布差异较大，显著性α=0.05
    * B_train 和 A_train的数据差异较大：有199维数据的分布相差大，所以考虑剔除掉199维数据
    * 缺失值少，且同分布的维度 极有可能是最重要的维度，可以尝试只取这部分数据进行分析， *可以做对比分析*



In [ ]:

# 使用ks_2samp检测分布差异，
dis_b_diff = {};dis_b_same={};dis_ab_diff={};dis_ab_same={};dis_abt_diff={};dis_abt_same={}
for column in a_b_less_null_columns.intersection(all_continuous_columns):
    # print column
    d, p = ks_2samp(b_data[column].dropna(), test_data[column].dropna())
    if p<=0.05:
        dis_b_diff[column] = (d,p)
    else:
        dis_b_same[column] = (d,p)
    d, p = ks_2samp(b_data[column].dropna(), a_data[column].dropna())
    if p<=0.05:
        dis_ab_diff[column] = (d,p)
    else:
        dis_ab_same[column] = (d,p)
    d, p = ks_2samp(test_data[column].dropna(), a_data[column].dropna())
    if p<=0.05:
        dis_abt_diff[column] = (d,p)
    else:
        dis_abt_same[column] = (d,p)
dis_b_diff = pd.DataFrame.from_dict(dis_b_diff,orient='index')
dis_b_same = pd.DataFrame.from_dict(dis_b_same,orient='index')
dis_ab_diff = pd.DataFrame.from_dict(dis_ab_diff,orient='index')
dis_ab_same = pd.DataFrame.from_dict(dis_ab_same,orient='index')

# 缺失值少，且同分布的维度
a_b_less_null_same_dis_columns = a_b_less_null_columns.intersection(dis_ab_same.index).append(pd.Index(['no','flag']))
#a_data = a_data[less_null_columns]
#b_data = b_data[less_null_columns]
#test_data = test_data[less_null_columns.drop(['flag'])]


* 再次离散化

In [ ]:
# 如何填充null值？？
# imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
# imp_mean.fit(a_data[all_continuous_columns])
# a_data[all_continuous_columns] = pd.DataFrame(imp_mean.transform(a_data[all_continuous_columns]),columns=all_continuous_columns)
# imp = IterativeImputer(max_iter=10, random_state=0)
# imp.fit(a_data[all_continuous_columns])
# b_data[all_continuous_columns] = pd.DataFrame(data=imp.transform(b_data[all_continuous_columns]),columns=all_continuous_columns)
# test_data['flag'] = np.nan
# test_data[all_continuous_columns] = pd.DataFrame(data=imp.transform(test_data[all_continuous_columns]),columns=all_continuous_columns)
# a_data[all_discrete_columns] = a_data[all_discrete_columns].fillna(1)
# b_data[all_discrete_columns] = b_data[all_discrete_columns].fillna(1)
# test_data[all_discrete_columns] = test_data[all_discrete_columns].fillna(1)
# a_data[all_discrete_columns] = a_data[all_discrete_columns].fillna(a_data[all_discrete_columns].mean())
# b_data[all_discrete_columns] = b_data[all_discrete_columns].fillna(a_data[all_discrete_columns].mean())
# test_data[all_discrete_columns] = test_data[all_discrete_columns].fillna(a_data[all_discrete_columns].mean())

# 离散化数据
threshold_k_bins = 0.1
kbd = KBinsDiscretizer(n_bins=np.round((b_fillna_data[b_continuous_columns].max()-b_fillna_data[b_continuous_columns].min())*threshold_k_bins).values,
                       encode='ordinal', strategy='uniform').fit(b_fillna_data[b_continuous_columns])
b_fillna_discrete_data = b_fillna_data
test_fillna_discrete_data = test_fillna_data
b_fillna_discrete_data[b_continuous_columns] = pd.DataFrame(kbd.transform(b_fillna_data[b_continuous_columns]),columns=b_continuous_columns)
test_fillna_discrete_data[b_continuous_columns] = pd.DataFrame(kbd.transform(test_fillna_data[b_continuous_columns]),columns=b_continuous_columns)



* 异常值分析
    * https://www.cnblogs.com/tinglele527/p/11955103.html
    * https://scikit-learn.org/stable/modules/outlier_detection.html
    * https://scikit-learn.org/0.20/auto_examples/plot_anomaly_comparison.html
    * https://blog.csdn.net/PbGc396Dwxjb77F2je/article/details/99687952
    * 离散值中取值比例很小的这部分，可能有两种情况：对预测结果有强作用，对预测情况无影响，*可以做对比分析*
    * 离散值中，取值比例很小的部分，如果熵很大，说明本身对结果没有区分度，这部分异常值可能性很大，
    * 离散值中，条件熵大的维度，区分度小，这部分维度可能需要去除掉
    * 经过前面的同分布处理后，条件熵减少了很多，
    * 连续值中，box plot可以很方便观测处异常值
    * 连续值在A_train中异常值偏少，整体少于4%，是否需要处理？连续值在B_train中，整体异常值少于2.5%
    *

In [ ]:
#标准差法 sunspots.counts > xbar + 2 * xstd
a_continuous_data = a_data[all_continuous_columns]
a_data_cont_abnormal = a_continuous_data > a_continuous_data.mean()+2*a_continuous_data.std()
a_data_cont_abnormal_proportion = np.sum(a_data_cont_abnormal, axis=0)/a_data_cont_abnormal.shape[0]
plt.hist(a_data_cont_abnormal_proportion, cumulative=True, bins=100, density=True);plt.show()

b_data_continuous = b_data[all_continuous_columns]
b_data_cont_abnormal = b_data_continuous > b_data_continuous.mean()+2*b_data_continuous.std()
b_data_cont_abnormal_proportion = np.sum(b_data_cont_abnormal, axis=0)/b_data_cont_abnormal.shape[0]
plt.hist(b_data_cont_abnormal_proportion, cumulative=True, bins=100, density=True);plt.show()

# 维度与熵
entropy_dis = {}
count_threshold = a_data.shape[0] * 0.05
entropy_threshold = 0.5
# 每个维度的异常熵
least_count_entropy = {}
for column in all_discrete_columns:
    s,d = conditional_entropy(a_data, cond=column)
    entropy_dis[column] = s
    for v in d:
        # 离散值中，如果某个值的数量很少，单独存起来
        if d[v][1]<=count_threshold and d[v][0]>=entropy_threshold:
            if column not in least_count_entropy:
                least_count_entropy[column] = {}
            least_count_entropy[column][v] = d[v]
entropy_dis = pd.DataFrame.from_dict(entropy_dis, orient='index')
print (entropy_dis.max(),entropy_dis.min())
plt.hist(np.sort(entropy_dis, axis=0), bins=100, cumulative=True);plt.show()




* 经过以上分析，可以分布验证数据的处理情况
    * 排除null值多的维度
    * 取A B同分布维度，B_test + B_train = B
    * 剔除线性相关性强的维度
    * 数据离散化：sklearn
    * 排除条件熵大的维度
    * 填充null值：固定填充，根据分布填充
    * 数据规范化